In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from decouple import config
import pandas as pd 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Configuración del WebDriver
chrome_options = Options()
# Aquí puedes agregar más opciones si lo necesitas
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)
import time 

# Función para iniciar sesión en el sitio web
def login_to_site():
    driver.get("https://www.bumeran.com.pe/login?returnTo=/")
    user = config("USER_LKN")
    clave = config("PASS_LKN")
    wait.until(EC.presence_of_element_located((By.ID, "email"))).send_keys(user)
    wait.until(EC.presence_of_element_located((By.ID, "password"))).send_keys(clave)
    time.sleep(3)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//a[text()=" Acepto"]'))).click()
    wait.until(EC.element_to_be_clickable((By.ID, "ingresar"))).click()

# Función para extraer información de la página
def extract_data_from_page():
    table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'sc-kdPYzB.egIfIy')))
    table_rows = table.find_elements(By.TAG_NAME, 'div')
    data = []

    for row in table_rows:
        title = get_element_text(row, By.CLASS_NAME, "sc-fhiYOA.zGAkU")
        company = get_element_text(row, By.CLASS_NAME, "sc-flvzOl.kJsfXU")
        location = get_element_text(row, By.CLASS_NAME, 'sc-cRULEh.dNmbcO')
        link = get_element_attribute(row, By.CLASS_NAME, 'sc-jkPxnQ.ioakMb', 'href')
        data.append({"title": title, "company": company, "location": location, "Link": link})
    return pd.DataFrame(data)

# Funciones auxiliares para obtener texto o atributos de elementos
def get_element_text(element, by, value):
    elements = element.find_elements(by, value)
    return elements[0].text if elements else None

def get_element_attribute(element, by, value, attribute):
    elements = element.find_elements(by, value)
    return elements[0].get_attribute(attribute) if elements else None

# Principal
if __name__ == "__main__":
    login_to_site()

    datas = []
    base_url = 'https://www.bumeran.com.pe/empleos-busqueda-data.html'
    
    for i in range(5):
        url = base_url if i == 0 else f'{base_url}?page={i+1}'
        driver.get(url)
        datas.append(extract_data_from_page())

    driver.quit()


In [6]:
info_df = pd.concat(datas, ignore_index=True)
info_df = info_df[info_df['Link'].str.len() > 5]


In [10]:
# Asumiendo que final_df ya está creado
palabras_clave = [ 
    "Datos",
    "Analista", 
    "Big Data", 
    "Scientist",
    "Científico",
    "Azure", 
    "Analyst", 
    "Analista",
    "engineer",
    "ingeniero"
]


palabras_excluir = [
    "Administrador", 
    "Arquitecto", 
    "Subgerente",
    "AWS",
    "Financiero",
    "Verificador",
    "Auxiliar",
    "Practicante",
    "Jefe",
    "Coordinador",
    "Operador"
]


regex_incluir = '|'.join(palabras_clave)
regex_excluir = '|'.join(palabras_excluir)

info_df = info_df[info_df['title'].str.contains(regex_incluir, case=False, na=False)]
info_df = info_df[~info_df['title'].str.contains(regex_excluir, case=False, na=False)]


In [11]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Asumiendo que final_df_filtrado ya está creado

# Crea un nuevo libro de trabajo y una hoja de trabajo
wb = Workbook()
ws = wb.active

# Transfiere los datos del DataFrame a la hoja de trabajo, creando hiperlinks para la columna de links
for i, row in enumerate(dataframe_to_rows(info_df, index=False, header=True)):
    row = list(row)
    if i == 0:
        ws.append(row)
    else:
        ws.append(row[:-1])
        ws['D%d' % (i+1)].hyperlink = row[-1]
        ws['D%d' % (i+1)].style = 'Hyperlink'

# Guarda el archivo
wb.save('Trabajo.xlsx')
